# What we're going to cover

- Downloading a text dataset
- Visualizing text data
- Converting text into numbers using tokenization
- Turning our tokenized text into an embedding
- Modelling a text dataset
    - Starting with a baseline (TF-IDF)
    - Building several deep learning text models
        - Dense, LSTM, GRU, Conv1D, Transfer learning
- Comparing the performance of each our models
- Combining our models into an ensemble
- Saving and loading a trained model
- Find the most wrong predictions

In [1]:
import datetime
print(f"Notebook last run (end-to-end): {datetime.datetime.now()}")

Notebook last run (end-to-end): 2025-12-16 23:08:13.413418


# Check for GPU

In [2]:
# Check for GPU
!nvidia-smi -L

GPU 0: NVIDIA T500 (UUID: GPU-2e388ea4-5136-87e9-1d14-8e74849e1774)


# Get helper functions

In [4]:
# Download helper functions script
#!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py

import urllib.request

url = "https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py"
urllib.request.urlretrieve(url, "helper_functions.py")


('helper_functions.py', <http.client.HTTPMessage at 0x2538e681c10>)

In [6]:
# Import series of helper functions for the notebook
from helper_functions import unzip_data, create_tensorboard_callback, plot_loss_curves, compare_historys

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
pip install tensorflow